# 1.1a: Extract Unembedding Matrix W

This notebook extracts the unembedding matrix **W** from a language model and saves it in its native bfloat16 format.

## What is W?

In a transformer language model, the **unembedding matrix** (also called `lm_head` in many implementations) maps the model's final hidden states back to vocabulary logits. For a model with:
- Vocabulary size V (number of tokens)
- Hidden dimension d (size of internal representations)

W is a V × d matrix where each row represents one token as a d-dimensional vector.

## Coordinate System: Gamma (γ)

We call this representation **gamma space** (γ): the origin-centered coordinate frame aligned with the model's natural axes. This is the raw geometric structure as trained.

Later notebooks may introduce:
- **γ'** (gamma prime): centroid-subtracted coordinates
- Other derived coordinate systems

But we start here with the pristine matrix as the model contains it.

## Why Save as bfloat16?

Modern models train in bfloat16 for efficiency. We preserve this:
1. **Scientific integrity**: This is the actual representation the model uses
2. **Enables quantization analysis**: Later we'll examine the discrete lattice structure
3. **Storage efficiency**: Half the size of float32

Individual notebooks will convert to float32 explicitly when needed for numerical precision.

## Parameters

In [1]:
# Model to extract from (Hugging Face identifier)
MODEL_ID = "Qwen/Qwen3-4B-Instruct-2507"
# MODEL_ID = "Qwen/Qwen2.5-3B-Instruct"
# MODEL_ID = "Qwen/Qwen1.5-4B-Chat"
# MODEL_ID = "unsloth/Llama-3.2-3B-Instruct"
# MODEL_ID = "google/gemma-3-4b-it"
# MODEL_ID = "ibm-granite/granite-4.0-micro"
# MODEL_ID = "microsoft/Phi-3-mini-4k-instruct"

# Output name (drop organization prefix for cleaner paths)
MODEL_NAME = "Qwen3-4B-Instruct-2507"

## Imports

In [2]:
import torch
from transformers import AutoModelForCausalLM
from safetensors.torch import save_file
from pathlib import Path

## Load Model and Extract W

In [3]:
print(f"Loading model: {MODEL_ID}")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.bfloat16,  # Load in native precision
    device_map="cpu",            # Keep on CPU (we're just extracting weights)
)

print(f"Model loaded successfully")

Loading model: Qwen/Qwen3-4B-Instruct-2507


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Model loaded successfully


In [4]:
# Extract unembedding matrix W
W = model.lm_head.weight.data

print(f"Unembedding matrix W:")
print(f"  Shape: {W.shape} (vocab_size × hidden_dim)")
print(f"  Dtype: {W.dtype}")
print(f"  Device: {W.device}")
print(f"  Memory: {W.element_size() * W.nelement() / 1024**2:.1f} MB")

Unembedding matrix W:
  Shape: torch.Size([151936, 2560]) (vocab_size × hidden_dim)
  Dtype: torch.bfloat16
  Device: cpu
  Memory: 741.9 MB


## Save W to Disk

We save in safetensors format (fast, safe, cross-platform) to `../tensors/{model_name}/W.safetensors`.

In [5]:
# Create output directory
output_dir = Path(f"../tensors/{MODEL_NAME}")
output_dir.mkdir(parents=True, exist_ok=True)

# Save W in pristine bfloat16 format
output_path = output_dir / "W.safetensors"
save_file({"W": W}, output_path)

print(f"✓ Saved W to {output_path}")
print(f"  File size: {output_path.stat().st_size / 1024**2:.1f} MB")

✓ Saved W to ../tensors/Qwen3-4B-Instruct-2507/W.safetensors
  File size: 741.9 MB


## Summary

We've extracted the unembedding matrix W and saved it in its native bfloat16 format. This matrix represents the vocabulary as a point cloud in high-dimensional space—each token is a vector.

**Next step:** Compute geometric properties (norms, distances, etc.) to begin exploring this space.